In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import seaborn as sns
import time
from ipywidgets import widgets
from IPython.display import display, clear_output
from IPython.display import display, clear_output
import ipywidgets as widgets
import random
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import datetime

In [2]:
big_model = tf.keras.models.load_model('my_model')
small_model = tf.keras.models.load_model('model_testowy')
# smaller_model= tf.keras.models.load_model('model_testowy')
# sprawdzenie architektury

In [3]:
import cv2
import cvzone
import os
import random
from cvzone.SelfiSegmentationModule import SelfiSegmentation
# Helper function used for visualization in the following examples
def identify_axes(ax_dict, fontsize=48):
    """
    Helper to identify the Axes in the examples below.

    Draws the label in a large font in the center of the Axes.

    Parameters
    ----------
    ax_dict : dict[str, Axes]
        Mapping between the title / label and the Axes.
    fontsize : int, optional
        How big the label should be.
    """
    kw = dict(ha="center", va="center", fontsize=fontsize, color="darkgrey")
#     for k, ax in ax_dict.items():
#         ax.text(0.5, 0.5, k, transform=ax.transAxes, **kw)

def make_live_prediction(model,external_camera=True,to_save=True,taca=False):
    df=pd.DataFrame(columns=['path','prediction'])
    print('\033[1mTo make photo and predictions click "space",to quit click "esc"\033[0m')

    if external_camera:
        cap = cv2.VideoCapture("/dev/video2")
    else:
        cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)
    # cap.set(cv2.CAP_PROP_FPS, 60)
    img_counter = 0
    if taca:
        segmentor = SelfiSegmentation()
        imgBG = cv2.imread("taca.png")
        imgBG=cv2.resize(imgBG,(640,480))

    while True:
        success, img = cap.read()
        if not success:
            print("failed to grab frame")
            break
        if taca is True:
            imgOut = segmentor.removeBG(img, imgBG, threshold=0.7)
        else:
            imgOut=img


    #     imgStack = cvzone.stackImages([img, imgOut], 2,1)
        cv2.imshow("image", imgOut) 
        k = cv2.waitKey(1)
        if k%256 == 27:
            # ESC pressed
            df=df.sort_index()
            display(df)
            break
        elif k%256 == 32:
            # SPACE pressed
            img_name = f"prediction_{img_counter}.png"
            if to_save:
                ct = datetime.datetime.now()
                os.mkdir(f"/home/home/Desktop/jdszr5-nonamezzz/Projekt_DL/predictions/{ct}")
                os.chdir(f"/home/home/Desktop/jdszr5-nonamezzz/Projekt_DL/predictions/{ct}")
                cv2.imwrite(img_name, imgOut)
                print("{} written!".format(img_name))
                os.chdir("/home/home/Desktop/jdszr5-nonamezzz/Projekt_DL/")
            img=cv2.cvtColor(imgOut, cv2.COLOR_BGR2RGB)
            cropped_img=img[0:480, 160:640]
            to_show=cv2.resize(cropped_img,(258,258))
            to_pred=(to_show.reshape(-1, 258, 258, 3))/255
            pred=model.predict(to_pred)
            result = np.where(pred[0] == np.amax(pred[0]))
            if result[0][0]==0:
                res='Apple A'
            elif result[0][0]==1:
                res='Apple B'
            elif result[0][0]==2:
                res='Apple C'
            elif result[0][0]==3:
                res='Apple D'
            elif result[0][0]==4:
                res='Apple E'
            else:
                res='Apple F'
            path=f"/home/home/Desktop/jdszr5-nonamezzz/Projekt_DL/Apple/{res}/"
            apples=random.sample(os.listdir(path),4)
            os.chdir(path)
#             comp=random.sample((os.listdir("/home/home/Desktop/jdszr5-nonamezzz/Projekt_DL/Apple/Apple A/Apple 1.png"), 4)
            comp1 = cv2.cvtColor(cv2.imread(apples[0]), cv2.COLOR_BGR2RGB)
            comp2 = cv2.cvtColor(cv2.imread(apples[1]), cv2.COLOR_BGR2RGB)
            comp3 = cv2.cvtColor(cv2.imread(apples[2]), cv2.COLOR_BGR2RGB)
            comp4 = cv2.cvtColor(cv2.imread(apples[3]), cv2.COLOR_BGR2RGB)
            df.loc[-1]=[img_name,res]
            df.index=df.index+1
            ax = plt.figure(figsize=(14, 7),constrained_layout=True).subplot_mosaic("""AABC
                                                                                       AADE""")
            identify_axes(ax)

            ax['A'].imshow(to_show)
#             ax['comp'].axis('off')  # clear x-axis and y-axis
            im = ax['B'].imshow(cv2.resize(comp1,(258,258)), cmap=plt.cm.hot, origin='upper')
            ax['B'].set_title('to compare_1')
            im = ax['C'].imshow(cv2.resize(comp2,(258,258)), cmap=plt.cm.hot, origin='upper')
            ax['C'].set_title('to compare_2')
            im = ax['D'].imshow(cv2.resize(comp3,(258,258)), cmap=plt.cm.hot, origin='upper')
            ax['D'].set_title('to compare_3') 
            im = ax['E'].imshow(cv2.resize(comp4,(258,258)), cmap=plt.cm.hot, origin='upper')
            ax['E'].set_title('to compare_4')
            ax['A'].set_title('captured photo for prediction')
            plt.show()
            print(f'Predicted - {res}')
            img_counter += 1
            os.chdir("/home/home/Desktop/jdszr5-nonamezzz/Projekt_DL/")

    cap.release()

    cv2.destroyAllWindows()


In [4]:
#widgets
style = {'description_width': 'initial'}

widget_model = widgets.RadioButtons(
    options=[ ('big model',big_model),('small model',small_model)],
    value=big_model,
    style=style,
    disabled = False,
    description='Choose model:')

widget_camera = widgets.RadioButtons(
    options=[('External Camera',True), ('Laptop camera',False)],
    value=True,
    style=style,
    disabled = False,
    description='Choose Camera:')

widget_save = widgets.RadioButtons(
    options=[('Yes',True), ('No',False)],
    value=True,
    style=style,
    disabled = False,
    description='Save photo?')


widget_back= widgets.RadioButtons(
    options=[('Yes',True), ('No',False)],
    value=False,
    style=style,
    disabled = False,
    description='Replace background?')




In [5]:
#widget handler

values = {'model_rb': widget_model.value,
          'camera_rb': widget_camera.value,
          'save_rb': widget_save.value,
          'back_rb': widget_back.value}
def widgets_handler(model_rb_val,camera_rb_val, save_rb_val, back_rb_val):
    values['model_rb'] = model_rb_val
    values['camera_rb'] = camera_rb_val
    values['save_rb'] = save_rb_val
    values['back_rb']=back_rb_val

    
result=widgets.interactive(widgets_handler,
                 model_rb_val= widget_model,
                 camera_rb_val = widget_camera,
                 save_rb_val = widget_save,
                 back_rb_val=widget_back)


In [6]:

#run button
b = widgets.Button(description="Live prediction",
    disabled=False,
    button_style='info',
    tooltip='Click me',
    icon='check'
)

layout = widgets.Layout(width='auto', height='auto', grid_gap='10px')

out = widgets.Output()

c=widgets.Button(description='Clear output')

# display(result,b,c)
# display(out)
def on_button_clicked_c(c):
    with out:
        clear_output()

def on_button_clicked(b):
    with out:
        clear_output()
        external_camera=values['camera_rb']
        model=values['model_rb']
        save=values['save_rb']
        taca=values['back_rb']
        make_live_prediction(model,external_camera, save,taca)

b.on_click(on_button_clicked)
c.on_click(on_button_clicked_c)

# def button_callback(button):
#     make_live_prediction(external_camera=False)
# run_button.on_click(button_callback)
controls1 = widgets.HBox(result.children[:2], layout=layout)
controls2 = widgets.HBox(result.children[2:], layout=layout)
buttons=widgets.HBox((b,c),layout=layout)

display(widgets.VBox([controls1,
                      controls2,
                      buttons,
                      out],layout=widgets.Layout(margin = '40px 20px 40px 20px',grid_gap='30px')))
